---
## 1. Dependencies

First, we need to implement all the building blocks from previous modules:

In [1]:
import json
import re
from typing import Any, Dict, Union, Optional, List
from pathlib import Path
from dataclasses import dataclass, field, asdict

print("✓ Standard library imports complete")

✓ Standard library imports complete


---
## 2. Exception Classes

Our exception hierarchy:

In [2]:
class Json2ToonError(Exception):
    """Base exception for json2toon errors."""
    pass


class EncodingError(Json2ToonError):
    """Raised when encoding fails."""
    pass


class DecodingError(Json2ToonError):
    """Raised when decoding fails."""
    pass


class AnalysisError(Json2ToonError):
    """Raised when analysis fails."""
    pass


class ConfigurationError(Json2ToonError):
    """Raised when configuration is invalid."""
    pass


print("✓ Exception classes defined")

✓ Exception classes defined


---
## 3. ToonConfig

Configuration dataclass with all options:

In [3]:
@dataclass
class ToonConfig:
    """
    Configuration for TOON encoding/decoding.
    
    All options with their defaults:
    """
    # Indentation
    indent_size: int = 2
    use_tabs: bool = False
    
    # Formatting
    quote_strings: bool = False
    trailing_newline: bool = True
    compact_arrays: bool = True
    
    # Tables
    use_table_format: bool = True
    table_min_rows: int = 2
    table_separator: str = " | "
    
    # Decoding
    strict_mode: bool = False
    preserve_order: bool = True


def get_default_config() -> ToonConfig:
    """Get the default configuration."""
    return ToonConfig()


print("✓ ToonConfig defined")
print(f"Default config: {get_default_config()}")

✓ ToonConfig defined
Default config: ToonConfig(indent_size=2, use_tabs=False, quote_strings=False, trailing_newline=True, compact_arrays=True, use_table_format=True, table_min_rows=2, table_separator=' | ', strict_mode=False, preserve_order=True)


---
## 4. Structure Analyzer

Helper functions for analyzing data structures:

In [4]:
@dataclass
class StructureInfo:
    """Information about a data structure."""
    is_uniform_array: bool = False
    array_keys: List[str] = field(default_factory=list)
    depth: int = 0
    has_nested_objects: bool = False
    has_nested_arrays: bool = False


def is_uniform_array(arr: list) -> tuple:
    """
    Check if array contains uniform dictionaries.
    
    Returns:
        (is_uniform, keys) tuple
    """
    if not arr or not all(isinstance(item, dict) for item in arr):
        return False, []
    
    first_keys = set(arr[0].keys())
    if not first_keys:
        return False, []
    
    for item in arr[1:]:
        if set(item.keys()) != first_keys:
            return False, []
    
    return True, list(first_keys)


def should_use_table_format(arr: list, config: ToonConfig) -> bool:
    """
    Determine if array should use table format.
    """
    if not config.use_table_format:
        return False
    
    if len(arr) < config.table_min_rows:
        return False
    
    is_uniform, keys = is_uniform_array(arr)
    if not is_uniform:
        return False
    
    # Don't use table for nested structures
    for item in arr:
        for value in item.values():
            if isinstance(value, (dict, list)):
                return False
    
    return True


print("✓ Analyzer functions defined")

✓ Analyzer functions defined


---
## 5. ToonEncoder (Complete)

Full encoder implementation:

In [5]:
class ToonEncoder:
    """
    Encoder that converts Python data to TOON format.
    """
    
    def __init__(self, config: Optional[ToonConfig] = None):
        self.config = config or get_default_config()
        self._indent_char = "\t" if self.config.use_tabs else " "
        self._indent_unit = self._indent_char * (1 if self.config.use_tabs else self.config.indent_size)
    
    def encode(self, data: Any) -> str:
        """Encode data to TOON format."""
        try:
            result = self._encode_value(data, 0)
            if self.config.trailing_newline and result and not result.endswith('\n'):
                result += '\n'
            return result
        except Exception as e:
            raise EncodingError(f"Encoding failed: {e}") from e
    
    def _looks_like_number(self, s: str) -> bool:
        """Check if string looks like a number."""
        if not s:
            return False
        try:
            float(s)
            return True
        except ValueError:
            return False
    
    def _encode_value(self, value: Any, indent: int) -> str:
        """Encode any value."""
        if value is None:
            return "null"
        elif isinstance(value, bool):
            return "true" if value else "false"
        elif isinstance(value, (int, float)):
            if isinstance(value, float):
                if value != value:  # NaN
                    return "null"
                if value == float('inf'):
                    return "null"
                if value == float('-inf'):
                    return "null"
            return str(value)
        elif isinstance(value, str):
            return self._encode_string(value)
        elif isinstance(value, list):
            return self._encode_array(value, indent)
        elif isinstance(value, dict):
            return self._encode_object(value, indent)
        else:
            return str(value)
    
    def _encode_string(self, s: str) -> str:
        """Encode a string value."""
        needs_quotes = (
            self.config.quote_strings or
            not s or
            s in ('true', 'false', 'null') or
            self._looks_like_number(s) or
            any(c in s for c in ':[]{}\n\r\t') or
            s.startswith(' ') or s.endswith(' ') or
            s.startswith('"') or s.startswith("'")
        )
        
        if needs_quotes:
            escaped = s.replace('\\', '\\\\').replace('"', '\\"').replace('\n', '\\n').replace('\r', '\\r').replace('\t', '\\t')
            return f'"{escaped}"'
        return s
    
    def _encode_array(self, arr: list, indent: int) -> str:
        """Encode an array."""
        if not arr:
            return "[]"
        
        # Check for table format
        if should_use_table_format(arr, self.config):
            return self._encode_table(arr, indent)
        
        # Check if compact format works
        if self.config.compact_arrays and all(
            isinstance(x, (int, float, str, bool, type(None))) and
            not (isinstance(x, str) and ('\n' in x or len(x) > 50))
            for x in arr
        ):
            items = [self._encode_value(x, indent) for x in arr]
            compact = f"[{', '.join(items)}]"
            if len(compact) <= 80:
                return compact
        
        # Check for nested arrays
        if all(isinstance(x, list) for x in arr):
            # Fallback to JSON for arrays of arrays
            return json.dumps(arr)
        
        # List format with dashes
        lines = []
        prefix = self._indent_unit * indent
        for item in arr:
            if isinstance(item, dict) and item:
                first_line = True
                for key, value in item.items():
                    encoded = self._encode_value(value, indent + 1)
                    if first_line:
                        lines.append(f"{prefix}- {key}: {encoded}")
                        first_line = False
                    else:
                        lines.append(f"{prefix}  {key}: {encoded}")
            else:
                encoded = self._encode_value(item, indent + 1)
                lines.append(f"{prefix}- {encoded}")
        
        return "\n" + "\n".join(lines) if indent > 0 else "\n".join(lines)
    
    def _encode_table(self, arr: list, indent: int) -> str:
        """Encode array as table."""
        keys = list(arr[0].keys())
        sep = self.config.table_separator
        
        lines = []
        prefix = self._indent_unit * indent
        
        # Header
        header = sep.join(keys)
        lines.append(f"{prefix}{header}")
        
        # Rows
        for item in arr:
            values = [self._encode_value(item.get(k), indent) for k in keys]
            row = sep.join(values)
            lines.append(f"{prefix}{row}")
        
        return "\n" + "\n".join(lines) if indent > 0 else "\n".join(lines)
    
    def _encode_object(self, obj: dict, indent: int) -> str:
        """Encode an object."""
        if not obj:
            return "{}"
        
        lines = []
        prefix = self._indent_unit * indent
        
        for key, value in obj.items():
            if isinstance(value, dict) and value:
                lines.append(f"{prefix}{key}:")
                nested = self._encode_object(value, indent + 1)
                lines.append(nested)
            elif isinstance(value, list):
                encoded = self._encode_array(value, indent + 1)
                if encoded.startswith('\n'):
                    lines.append(f"{prefix}{key}:{encoded}")
                else:
                    lines.append(f"{prefix}{key}: {encoded}")
            else:
                encoded = self._encode_value(value, indent)
                lines.append(f"{prefix}{key}: {encoded}")
        
        return "\n".join(lines)


print("✓ ToonEncoder defined")

✓ ToonEncoder defined


---
## 6. ToonDecoder (Complete)

Full decoder implementation:

In [6]:
class ToonDecoder:
    """
    Decoder that converts TOON format back to Python data.
    """
    
    def __init__(self, config: Optional[ToonConfig] = None):
        self.config = config or get_default_config()
    
    def decode(self, toon_str: str) -> Any:
        """Decode TOON string to Python data."""
        try:
            if not toon_str or not toon_str.strip():
                return {}
            
            lines = toon_str.split('\n')
            return self._parse_lines(lines, 0)[0]
        except Exception as e:
            raise DecodingError(f"Decoding failed: {e}") from e
    
    def _get_indent(self, line: str) -> int:
        """Get indentation level."""
        stripped = line.lstrip()
        if not stripped:
            return 0
        indent_chars = len(line) - len(stripped)
        return indent_chars // self.config.indent_size
    
    def _parse_lines(self, lines: List[str], start_indent: int) -> tuple:
        """Parse lines into a data structure."""
        result = {}
        i = 0
        
        while i < len(lines):
            line = lines[i]
            if not line.strip():
                i += 1
                continue
            
            current_indent = self._get_indent(line)
            stripped = line.strip()
            
            if current_indent < start_indent:
                break
            
            # List item
            if stripped.startswith('- '):
                list_result, consumed = self._parse_list(lines[i:], current_indent)
                return list_result, i + consumed
            
            # Table format
            if self.config.table_separator in stripped and ':' not in stripped:
                table_result, consumed = self._parse_table(lines[i:], current_indent)
                return table_result, i + consumed
            
            # Key-value pair
            if ':' in stripped:
                colon_pos = stripped.find(':')
                key = stripped[:colon_pos].strip()
                value_part = stripped[colon_pos + 1:].strip()
                
                if value_part:
                    result[key] = self._parse_value(value_part)
                else:
                    # Nested structure
                    nested_lines = []
                    j = i + 1
                    while j < len(lines):
                        if lines[j].strip() and self._get_indent(lines[j]) <= current_indent:
                            break
                        nested_lines.append(lines[j])
                        j += 1
                    
                    if nested_lines:
                        nested_result, _ = self._parse_lines(nested_lines, current_indent + 1)
                        result[key] = nested_result
                    else:
                        result[key] = {}
                    
                    i = j
                    continue
            
            i += 1
        
        return result, i
    
    def _parse_value(self, value_str: str) -> Any:
        """Parse a value string."""
        value_str = value_str.strip()
        
        if not value_str:
            return ""
        
        # Null
        if value_str == 'null':
            return None
        
        # Booleans
        if value_str == 'true':
            return True
        if value_str == 'false':
            return False
        
        # JSON array
        if value_str.startswith('['):
            try:
                return json.loads(value_str)
            except json.JSONDecodeError:
                return value_str
        
        # JSON object
        if value_str.startswith('{'):
            try:
                return json.loads(value_str)
            except json.JSONDecodeError:
                return value_str
        
        # Quoted string
        if (value_str.startswith('"') and value_str.endswith('"')) or \
           (value_str.startswith("'") and value_str.endswith("'")):
            inner = value_str[1:-1]
            # Handle escape sequences
            inner = inner.replace('\\n', '\n').replace('\\r', '\r').replace('\\t', '\t')
            inner = inner.replace('\\"', '"').replace("\\'" , "'")
            inner = inner.replace('\\\\', '\\')
            return inner
        
        # Number
        try:
            if '.' in value_str or 'e' in value_str.lower():
                return float(value_str)
            return int(value_str)
        except ValueError:
            pass
        
        # Plain string
        return value_str
    
    def _parse_list(self, lines: List[str], base_indent: int) -> tuple:
        """Parse a list structure."""
        result = []
        i = 0
        
        while i < len(lines):
            line = lines[i]
            if not line.strip():
                i += 1
                continue
            
            current_indent = self._get_indent(line)
            stripped = line.strip()
            
            if current_indent < base_indent:
                break
            
            if current_indent == base_indent and stripped.startswith('- '):
                item_content = stripped[2:].strip()
                
                if ':' in item_content:
                    # Object item
                    obj = {}
                    colon_pos = item_content.find(':')
                    key = item_content[:colon_pos].strip()
                    value = item_content[colon_pos + 1:].strip()
                    obj[key] = self._parse_value(value)
                    
                    # Check for continuation
                    j = i + 1
                    while j < len(lines):
                        next_line = lines[j]
                        if not next_line.strip():
                            j += 1
                            continue
                        next_indent = self._get_indent(next_line)
                        next_stripped = next_line.strip()
                        
                        if next_indent <= base_indent:
                            break
                        
                        if ':' in next_stripped:
                            colon_pos = next_stripped.find(':')
                            k = next_stripped[:colon_pos].strip()
                            v = next_stripped[colon_pos + 1:].strip()
                            obj[k] = self._parse_value(v)
                        j += 1
                    
                    result.append(obj)
                    i = j
                    continue
                else:
                    result.append(self._parse_value(item_content))
            
            i += 1
        
        return result, i
    
    def _parse_table(self, lines: List[str], base_indent: int) -> tuple:
        """Parse a table structure."""
        result = []
        sep = self.config.table_separator
        
        header_line = lines[0].strip()
        headers = [h.strip() for h in header_line.split(sep)]
        
        i = 1
        while i < len(lines):
            line = lines[i]
            if not line.strip():
                i += 1
                continue
            
            if self._get_indent(line) < base_indent:
                break
            
            stripped = line.strip()
            if sep in stripped:
                values = [v.strip() for v in stripped.split(sep)]
                row = {}
                for j, header in enumerate(headers):
                    if j < len(values):
                        row[header] = self._parse_value(values[j])
                    else:
                        row[header] = None
                result.append(row)
            else:
                break
            
            i += 1
        
        return result, i


print("✓ ToonDecoder defined")

✓ ToonDecoder defined


---
## 7. Metrics (Simplified)

For the core module, we include a simplified metrics system:

In [7]:
try:
    import tiktoken
    HAS_TIKTOKEN = True
except ImportError:
    HAS_TIKTOKEN = False
    print("⚠ tiktoken not installed - using character-based estimation")


@dataclass
class ComparisonResult:
    """Results of comparing JSON vs TOON formats."""
    json_tokens: int
    toon_tokens: int
    savings: int
    savings_percent: float


def count_tokens(text: str, encoding_name: str = "cl100k_base") -> int:
    """Count tokens in text."""
    if HAS_TIKTOKEN:
        encoding = tiktoken.get_encoding(encoding_name)
        return len(encoding.encode(text))
    else:
        # Rough estimation: ~4 chars per token
        return len(text) // 4


def compare_formats(data: Any, encoder: ToonEncoder, encoding_name: str = "cl100k_base") -> ComparisonResult:
    """Compare token counts between JSON and TOON."""
    json_str = json.dumps(data, indent=2)
    json_tokens = count_tokens(json_str, encoding_name)
    
    toon_str = encoder.encode(data)
    toon_tokens = count_tokens(toon_str, encoding_name)
    
    savings = json_tokens - toon_tokens
    savings_percent = (savings / json_tokens * 100) if json_tokens > 0 else 0
    
    return ComparisonResult(
        json_tokens=json_tokens,
        toon_tokens=toon_tokens,
        savings=savings,
        savings_percent=savings_percent
    )


def generate_report(comparison: ComparisonResult, output_format: str = "text") -> str:
    """Generate a comparison report."""
    if output_format == "json":
        return json.dumps({
            "json_tokens": comparison.json_tokens,
            "toon_tokens": comparison.toon_tokens,
            "savings": comparison.savings,
            "savings_percent": round(comparison.savings_percent, 2)
        }, indent=2)
    elif output_format == "markdown":
        return f"""# Token Comparison Report

| Format | Tokens |
|--------|--------|
| JSON   | {comparison.json_tokens} |
| TOON   | {comparison.toon_tokens} |

**Savings:** {comparison.savings} tokens ({comparison.savings_percent:.1f}%)
"""
    else:
        return f"""Token Comparison Report:
- JSON tokens: {comparison.json_tokens}
- TOON tokens: {comparison.toon_tokens}
- Savings: {comparison.savings} tokens ({comparison.savings_percent:.1f}%)"""


print("✓ Metrics functions defined")

✓ Metrics functions defined


---
## 8. Core Convenience Functions

Now we implement the main convenience functions that users will typically use:

### 8.1 json_to_toon()

The simplest way to convert JSON data to TOON:

In [8]:
def json_to_toon(
    data: Any,
    config: Optional[ToonConfig] = None
) -> str:
    """
    Convert JSON data to TOON format.
    
    This is the primary convenience function for encoding.
    
    Args:
        data: Python data structure (dict, list, primitives)
        config: Optional ToonConfig for customization
        
    Returns:
        TOON-formatted string
        
    Example:
        >>> toon = json_to_toon({"name": "Alice", "age": 30})
        >>> print(toon)
        name: Alice
        age: 30
    """
    # Create encoder with provided or default config
    encoder = ToonEncoder(config or get_default_config())
    
    # Encode and return
    return encoder.encode(data)


print("✓ json_to_toon() defined")

✓ json_to_toon() defined


### Testing json_to_toon()

In [9]:
# Simple example
data = {"name": "Alice", "age": 30, "active": True}
toon = json_to_toon(data)

print("Input (Python dict):")
print(data)
print()
print("Output (TOON):")
print(toon)

Input (Python dict):
{'name': 'Alice', 'age': 30, 'active': True}

Output (TOON):
name: Alice
age: 30
active: true



In [10]:
# With custom config
custom_config = ToonConfig(
    quote_strings=True,
    indent_size=4
)

toon_custom = json_to_toon(data, custom_config)
print("With custom config (quote_strings=True):")
print(toon_custom)

With custom config (quote_strings=True):
name: "Alice"
age: 30
active: true



### 8.2 toon_to_json()

Convert TOON format back to Python data:

In [11]:
def toon_to_json(
    toon_str: str,
    config: Optional[ToonConfig] = None
) -> Any:
    """
    Convert TOON format to JSON data.
    
    This is the primary convenience function for decoding.
    
    Args:
        toon_str: TOON-formatted string
        config: Optional ToonConfig for customization
        
    Returns:
        Decoded Python data (dict, list, or primitive)
        
    Example:
        >>> toon = '''name: Alice
        ... age: 30'''
        >>> data = toon_to_json(toon)
        >>> print(data)  # {'name': 'Alice', 'age': 30}
    """
    # Create decoder with provided or default config
    decoder = ToonDecoder(config or get_default_config())
    
    # Decode and return
    return decoder.decode(toon_str)


print("✓ toon_to_json() defined")

✓ toon_to_json() defined


### Testing toon_to_json()

In [12]:
# Simple example
toon_str = """name: Alice
age: 30
active: true"""

data = toon_to_json(toon_str)

print("Input (TOON):")
print(toon_str)
print()
print("Output (Python dict):")
print(data)
print()
print("As JSON:")
print(json.dumps(data, indent=2))

Input (TOON):
name: Alice
age: 30
active: true

Output (Python dict):
{'name': 'Alice', 'age': 30, 'active': True}

As JSON:
{
  "name": "Alice",
  "age": 30,
  "active": true
}


### 8.3 convert_file()

Convert files between JSON and TOON formats:

In [13]:
def convert_file(
    input_path: Union[str, Path],
    output_path: Union[str, Path],
    to_toon: bool = True,
    config: Optional[ToonConfig] = None
) -> None:
    """
    Convert file between JSON and TOON formats.
    
    Args:
        input_path: Path to input file
        output_path: Path to output file
        to_toon: True for JSON→TOON, False for TOON→JSON
        config: Optional ToonConfig for customization
        
    Example:
        # Convert JSON to TOON
        convert_file('data.json', 'data.toon')
        
        # Convert TOON back to JSON
        convert_file('data.toon', 'data.json', to_toon=False)
        
    Implementation:
        1. Convert paths to Path objects
        2. Read input file content
        3. Convert based on direction (to_toon flag)
        4. Write result to output file
    """
    # Ensure Path objects
    input_path = Path(input_path)
    output_path = Path(output_path)
    
    # Read input file
    with open(input_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Convert based on direction
    if to_toon:
        # JSON → TOON
        data = json.loads(content)
        result = json_to_toon(data, config)
    else:
        # TOON → JSON
        data = toon_to_json(content, config)
        result = json.dumps(data, indent=2)
    
    # Write output file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(result)


print("✓ convert_file() defined")

✓ convert_file() defined


### Testing convert_file()

In [19]:
# Create a temporary JSON file
test_data = {
    "users": [
        {"name": "Alice", "age": 30},
        {"name": "Bob", "age": 25}
    ],
    "count": 2
}

# Write test JSON file
with open('test_input.json', 'w') as f:
    json.dump(test_data, f, indent=2)

print("Created test_input.json:")
with open('test_input.json') as f:
    print(f.read())

Created test_input.json:
{
  "users": [
    {
      "name": "Alice",
      "age": 30
    },
    {
      "name": "Bob",
      "age": 25
    }
  ],
  "count": 2
}


In [20]:
# Convert to TOON
convert_file('test_input.json', 'test_output.toon', to_toon=True)

print("Converted to TOON (test_output.toon):")
with open('test_output.toon') as f:
    print(f.read())

Converted to TOON (test_output.toon):
users:
  name | age
  Alice | 30
  Bob | 25
count: 2



In [21]:
# Convert back to JSON
convert_file('test_output.toon', 'test_roundtrip.json', to_toon=False)

print("Converted back to JSON (test_roundtrip.json):")
with open('test_roundtrip.json') as f:
    print(f.read())

Converted back to JSON (test_roundtrip.json):
{
  "users": [
    {
      "name": "Alice",
      "age": 30
    },
    {
      "name": "Bob",
      "age": 25
    }
  ],
  "count": 2
}


In [17]:
# Cleanup
import os
for f in ['test_input.json', 'test_output.toon', 'test_roundtrip.json']:
    if os.path.exists(f):
        os.remove(f)
print("✓ Cleanup complete")

✓ Cleanup complete


### 8.4 get_conversion_stats()

Get statistics about conversion:

In [22]:
def get_conversion_stats(
    data: Any,
    config: Optional[ToonConfig] = None,
    output_format: str = "text"
) -> Dict[str, Any]:
    """
    Get statistics about JSON to TOON conversion.
    
    This function provides insights into the token savings
    achieved by converting to TOON format.
    
    Args:
        data: Python data to analyze
        config: Optional ToonConfig
        output_format: Report format ('text', 'json', 'markdown')
        
    Returns:
        Dictionary with conversion statistics:
        - json_tokens: Token count for JSON
        - toon_tokens: Token count for TOON
        - savings: Absolute token savings
        - savings_percent: Percentage savings
        - report: Formatted report string
        
    Example:
        >>> stats = get_conversion_stats({"name": "Alice"})
        >>> print(stats['savings_percent'])  # e.g., 35.5
    """
    # Create encoder
    encoder = ToonEncoder(config or get_default_config())
    
    # Get comparison metrics
    comparison = compare_formats(data, encoder)
    
    # Generate formatted report
    report = generate_report(comparison, output_format)
    
    # Return combined results
    return {
        "json_tokens": comparison.json_tokens,
        "toon_tokens": comparison.toon_tokens,
        "savings": comparison.savings,
        "savings_percent": comparison.savings_percent,
        "report": report
    }


print("✓ get_conversion_stats() defined")

✓ get_conversion_stats() defined


### Testing get_conversion_stats()

In [23]:
# Sample data
sample_data = {
    "company": "TechCorp",
    "employees": [
        {"name": "Alice", "role": "Developer", "years": 5},
        {"name": "Bob", "role": "Designer", "years": 3},
        {"name": "Charlie", "role": "Manager", "years": 8}
    ],
    "founded": 2010,
    "active": True
}

# Get stats
stats = get_conversion_stats(sample_data)

print("Conversion Statistics:")
print(f"  JSON tokens:    {stats['json_tokens']}")
print(f"  TOON tokens:    {stats['toon_tokens']}")
print(f"  Savings:        {stats['savings']} tokens")
print(f"  Savings %:      {stats['savings_percent']:.1f}%")
print()
print("Report:")
print(stats['report'])

Conversion Statistics:
  JSON tokens:    107
  TOON tokens:    49
  Savings:        58 tokens
  Savings %:      54.2%

Report:
Token Comparison Report:
- JSON tokens: 107
- TOON tokens: 49
- Savings: 58 tokens (54.2%)


In [24]:
# Markdown report
stats_md = get_conversion_stats(sample_data, output_format="markdown")
print(stats_md['report'])

# Token Comparison Report

| Format | Tokens |
|--------|--------|
| JSON   | 107 |
| TOON   | 49 |

**Savings:** 58 tokens (54.2%)



---
## 9. Complete Workflow Example

Let's demonstrate a complete workflow using all the core functions:

In [25]:
# Real-world example: API response data
api_response = {
    "status": "success",
    "data": {
        "products": [
            {"id": 1, "name": "Widget A", "price": 19.99, "stock": 100},
            {"id": 2, "name": "Widget B", "price": 29.99, "stock": 50},
            {"id": 3, "name": "Widget C", "price": 39.99, "stock": 25}
        ],
        "pagination": {
            "page": 1,
            "total": 3,
            "hasMore": False
        }
    },
    "timestamp": "2024-01-15T10:30:00Z"
}

print("=" * 60)
print("STEP 1: Original JSON Data")
print("=" * 60)
print(json.dumps(api_response, indent=2))

STEP 1: Original JSON Data
{
  "status": "success",
  "data": {
    "products": [
      {
        "id": 1,
        "name": "Widget A",
        "price": 19.99,
        "stock": 100
      },
      {
        "id": 2,
        "name": "Widget B",
        "price": 29.99,
        "stock": 50
      },
      {
        "id": 3,
        "name": "Widget C",
        "price": 39.99,
        "stock": 25
      }
    ],
    "pagination": {
      "page": 1,
      "total": 3,
      "hasMore": false
    }
  },
  "timestamp": "2024-01-15T10:30:00Z"
}


In [26]:
print("=" * 60)
print("STEP 2: Convert to TOON")
print("=" * 60)
toon_output = json_to_toon(api_response)
print(toon_output)

STEP 2: Convert to TOON
status: success
data:
  products:
    id | name | price | stock
    1 | Widget A | 19.99 | 100
    2 | Widget B | 29.99 | 50
    3 | Widget C | 39.99 | 25
  pagination:
    page: 1
    total: 3
    hasMore: false
timestamp: "2024-01-15T10:30:00Z"



In [27]:
print("=" * 60)
print("STEP 3: Get Conversion Stats")
print("=" * 60)
stats = get_conversion_stats(api_response, output_format="markdown")
print(stats['report'])

STEP 3: Get Conversion Stats
# Token Comparison Report

| Format | Tokens |
|--------|--------|
| JSON   | 175 |
| TOON   | 101 |

**Savings:** 74 tokens (42.3%)



In [28]:
print("=" * 60)
print("STEP 4: Convert Back to JSON (Round-trip)")
print("=" * 60)
recovered = toon_to_json(toon_output)
print(json.dumps(recovered, indent=2))

STEP 4: Convert Back to JSON (Round-trip)
{
  "status": "success",
  "data": {
    "products": [
      {
        "id": 1,
        "name": "Widget A",
        "price": 19.99,
        "stock": 100
      },
      {
        "id": 2,
        "name": "Widget B",
        "price": 29.99,
        "stock": 50
      },
      {
        "id": 3,
        "name": "Widget C",
        "price": 39.99,
        "stock": 25
      }
    ],
    "pagination": {
      "page": 1,
      "total": 3,
      "hasMore": false
    }
  },
  "timestamp": "2024-01-15T10:30:00Z"
}


In [29]:
print("=" * 60)
print("STEP 5: Verify Round-trip")
print("=" * 60)

# Compare original and recovered
if json.dumps(api_response, sort_keys=True) == json.dumps(recovered, sort_keys=True):
    print("✓ Round-trip successful! Data matches exactly.")
else:
    print("✗ Round-trip mismatch detected.")
    print("Original:", api_response)
    print("Recovered:", recovered)

STEP 5: Verify Round-trip
✓ Round-trip successful! Data matches exactly.


---
## 10. Summary

### Core Functions Implemented:

| Function | Purpose |
|----------|--------|
| `json_to_toon(data, config)` | Convert Python data to TOON string |
| `toon_to_json(toon_str, config)` | Convert TOON string to Python data |
| `convert_file(input, output, to_toon, config)` | Convert files between formats |
| `get_conversion_stats(data, config, format)` | Get token savings statistics |

### Key Design Decisions:

1. **Optional Config** - All functions accept optional config, using defaults if not provided
2. **Simple API** - Hide complexity of encoder/decoder instantiation
3. **Path Support** - `convert_file` accepts both strings and Path objects
4. **Flexible Output** - Stats can be returned in text, JSON, or markdown format

### Usage Patterns:

```python
# Quick conversion
toon = json_to_toon({"key": "value"})

# With custom config
config = ToonConfig(indent_size=4)
toon = json_to_toon(data, config)

# File conversion
convert_file("input.json", "output.toon")

# Get savings info
stats = get_conversion_stats(data)
print(f"Saved {stats['savings_percent']:.1f}% tokens")
```

In [30]:
print("✓ Module 7: Core - Complete!")
print()
print("Functions implemented:")
print("  - json_to_toon(data, config)")
print("  - toon_to_json(toon_str, config)")
print("  - convert_file(input_path, output_path, to_toon, config)")
print("  - get_conversion_stats(data, config, output_format)")
print()
print("This completes the json2toon implementation notebooks!")
print()
print("All 7 modules:")
print("  1. exceptions - Error handling")
print("  2. config     - Configuration")
print("  3. analyzer   - Structure analysis")
print("  4. encoder    - JSON→TOON conversion")
print("  5. decoder    - TOON→JSON conversion")
print("  6. metrics    - Token counting")
print("  7. core       - Convenience functions")

✓ Module 7: Core - Complete!

Functions implemented:
  - json_to_toon(data, config)
  - toon_to_json(toon_str, config)
  - convert_file(input_path, output_path, to_toon, config)
  - get_conversion_stats(data, config, output_format)

This completes the json2toon implementation notebooks!

All 7 modules:
  1. exceptions - Error handling
  2. config     - Configuration
  3. analyzer   - Structure analysis
  4. encoder    - JSON→TOON conversion
  5. decoder    - TOON→JSON conversion
  6. metrics    - Token counting
  7. core       - Convenience functions
